## Setup

In [ ]:
import os
import folium

In [ ]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

We will be using OpenRouteService API to calculate the directions. Please sign-up for a free account and create an API key. If you already have an account, the API key is obtained from the OpenRouteService Dashboard. Enter your API key below.

In [ ]:
ORS_API_KEY = '5b3ce3597851110001cf6248d629d5a21a0746fb99781645b0b016d1'

## Folium Basics

We will learn the basics of folium by creating an interactive map showing the driving directions between two chosen locations. Let’s start by defining the coordinates of two cities.

In [ ]:
san_francisco = (37.7749, -122.4194)
new_york = (40.661, -73.944)

In [ ]:
m = folium.Map()
m

In [ ]:
from folium import Figure
fig = Figure(width=800, height=400)
m = folium.Map(location=[39.83, -98.58], zoom_start=4)
fig.add_child(m)

The map object `m` can be manipulated by adding different elements to it. Contrary to how Matplotlib objects work, the map object does not get emptied when displayed. So you are able to visualize and incrementally add elements to it. Let’s add some markers to the map using `folium.map.Marker` class.



In [ ]:
folium.Marker(san_francisco, popup='San Francisco').add_to(m)
folium.Marker(new_york, popup='New York').add_to(m)
m

The markers can be customized to have a different color or icons. You can check the folium.map.Icon class for options for creating icons. This class supports a vast range of icons from the fontawesome icons and bootstrap icons libraries. You can choose the name of the icon from there to use it in your Folium map. The prefix parameter can be fa for FontAwesome icons or glyphicon for Bootstrap3.

In [ ]:
from folium import Figure
fig = Figure(width=800, height=400)

m = folium.Map(location=[39.83, -98.58], zoom_start = 4)

folium.Marker(san_francisco, popup = 'San Francisco',
              icon=folium.Icon(
                  color = 'green', icon = 'crosshairs', prefix = 'fa')
              ).add_to(m)

folium.Marker(new_york, popup='New York', 
              icon=folium.Icon(color='red', icon='crosshairs', prefix='fa')
             ).add_to(m)
fig.add_child(m)

In [ ]:
import requests
from pprint import pprint
import json

san_francisco = (37.7749, -122.4194)
new_york = (40.661, -73.944)

parameters = {
    'api_key' : ORS_API_KEY,
    'start' : '{}, {}'.format(san_francisco[1], san_francisco[0]),
    'end' : '{}, {}'.format(new_york[1], new_york[0])
}

response = requests.get('https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

if response.status_code == 200:
  print('Request Successful')
  data = response.json()
else:
  print('Request failed.')

Request Successful


In [ ]:
print(data)
# pprint(data)

{'type': 'FeatureCollection', 'features': [{'bbox': [-122.41945, 37.774939, -73.944026, 41.789337], 'type': 'Feature', 'properties': {'segments': [{'distance': 4692195.2, 'duration': 166697.1, 'steps': [{'distance': 248.5, 'duration': 33.1, 'type': 11, 'instruction': 'Head northeast on Market Street', 'name': 'Market Street', 'way_points': [0, 11]}, {'distance': 191.5, 'duration': 24.6, 'type': 1, 'instruction': 'Turn right onto 10th Street', 'name': '10th Street', 'way_points': [11, 16]}, {'distance': 1728.4, 'duration': 162.2, 'type': 0, 'instruction': 'Turn left onto Mission Street', 'name': 'Mission Street', 'way_points': [16, 58]}, {'distance': 189.1, 'duration': 17.0, 'type': 5, 'instruction': 'Turn slight right onto Mission Street', 'name': 'Mission Street', 'way_points': [58, 63]}, {'distance': 125.2, 'duration': 10.0, 'type': 1, 'instruction': 'Turn right onto New Montgomery Street', 'name': 'New Montgomery Street', 'way_points': [63, 66]}, {'distance': 88.6, 'duration': 21.3,

Extract the coordinates for the driving directions.

In [ ]:
route = data['features'][0]['geometry']['coordinates']
route[:5]

[[-122.41945, 37.774939],
 [-122.419433, 37.774953],
 [-122.41934, 37.77502],
 [-122.419254, 37.775097],
 [-122.419176, 37.775163]]

The coordinates returned by OpenRouteService API is in the order [X,Y] (i.e. [Longitude, Latitude]) whereas Folium requires the coordinates in [Y,X] (i.e. [Latitude, Longitude]) order. We can swap them before plotting.

In [ ]:
route_xy = []
for x, y in route:
  route_xy.append((y, x))
route_xy[:5]

# or we can use another method for the same action by using list comprehension
# route_xy = [(y,x) for x, y in route]
# route_xy[:5]

[(37.774939, -122.41945),
 (37.774953, -122.419433),
 (37.77502, -122.41934),
 (37.775097, -122.419254),
 (37.775163, -122.419176)]

We extract the route summary returned by the API which contains the total driving distance in meters.

In [ ]:
summary = data['features'][0]['properties']['summary']
distance = round(summary['distance']/1000)
tooltip = 'Driving Distance: {}km'.format(distance)

We can use the folium.vector_layers.Polyline class to add a line to the map. The class has a smooth_factor parameter which can be used to simplify the line displayed when zoomed-out. Setting a higher number results in better performance.

In [ ]:
folium.PolyLine(route_xy, tooltip=tooltip, smooth_factor=1).add_to(m)
fig.add_child(m)

In [ ]:
output_folder = 'output'

if not os.path.exists(output_folder):
  os.mkdir(output_folder)
output_path = os.path.join(output_folder, 'directions.html')
m.save(output_path)